# BOW

https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568

In [ ]:
import numpy as np
import pandas as pd

#Loading the data

In [ ]:
df1 = pd.read_excel("/content/FinalHinTrainTrac2_BackTran(clean).xlsx", index_col = 0)
df2 = pd.read_excel("/content/FinalHinDevTrac2_BackTran(clean).xlsx", index_col = 0)

In [ ]:
df1.head()

,ID,data,Sub-task A,Sub-task B
0,C4.131,बॉलीवुड फिल्म देखने के समय तर्क घर में चोरके ए...,NAG,NGEN
1,C4.638,चुटिया फिल्म,NAG,NGEN
2,C38.598,हमारे जाट बन्दे का खेने का अर्थ था मार्च डालुग...,OAG,NGEN
3,C4.2101.1,फेमिनिस्म कैंसर की नारीवादी ये साही हैं संयुक...,OAG,NGEN
4,C29.14.2,अमृत आनंद अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


In [ ]:
df2.head()

,ID,data,Sub-task A,Sub-task B
0,C38.9,बक्ल साक्षात्कारकर्ताओं कुछ जाड़ा हाय खुले दिम...,OAG,NGEN
1,C4.1510,भैया शादी मैं ज़रूर आना फिल्म का कथानक था क्या...,NAG,NGEN
2,C19.95,धारा हैं क्या ये बताते कम से कम को,NAG,NGEN
3,C4.281,कबीर सिंह मारा हुई इससे पता चलता है आजकल के लो...,OAG,NGEN
4,C4.216,मेन इतनी कम नापसंद क्भी न्ही देखी,NAG,NGEN


In [ ]:
df1['Sub-task A'].value_counts()

CAG    2453
NAG    2245
OAG    1820
Name: Sub-task A, dtype: int64

In [ ]:
df2['Sub-task A'].value_counts()

CAG    602
OAG    582
NAG    578
Name: Sub-task A, dtype: int64

In [ ]:
df = pd.concat([df1, df2], ignore_index = True)

In [ ]:
df.head()

,ID,data,Sub-task A,Sub-task B
0,C4.131,बॉलीवुड फिल्म देखने के समय तर्क घर में चोरके ए...,NAG,NGEN
1,C4.638,चुटिया फिल्म,NAG,NGEN
2,C38.598,हमारे जाट बन्दे का खेने का अर्थ था मार्च डालुग...,OAG,NGEN
3,C4.2101.1,फेमिनिस्म कैंसर की नारीवादी ये साही हैं संयुक...,OAG,NGEN
4,C29.14.2,अमृत आनंद अब तो जुड़े ही है उनको बोलो जुड़ने,NAG,NGEN


In [ ]:
samples = df['data']
labels = df['Sub-task A']

In [ ]:
print(len(samples))
print(len(labels))

8280
8280


# Shuffling the data and splitting it

In [ ]:
# Shuffle the data
seed = 1333
rng = np.random.RandomState(seed)
rng.shuffle(samples)
rng = np.random.RandomState(seed)
rng.shuffle(labels)

# Extract a training & validation split
validation_split = 0.2
num_validation_samples = int(validation_split * len(samples))
train_samples = samples[:-num_validation_samples]
val_samples = samples[-num_validation_samples:]
train_labels = labels[:-num_validation_samples]
val_labels = labels[-num_validation_samples:]

In [ ]:
train_labels = list(train_labels)
val_labels = list(val_labels)

In [ ]:
class_names = list(df['Sub-task A'].unique())
class_names

['CAG', 'OAG', 'NAG']

In [ ]:
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from tensorflow.keras.utils import to_categorical

In [ ]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_samples) # only fit on train

In [ ]:
x_train = tokenize.texts_to_matrix(train_samples)
x_test = tokenize.texts_to_matrix(val_samples)

# Changing the train and val labels from string to integers

In [ ]:
encoder = LabelEncoder()
encoder.fit(train_labels)
y_train = encoder.transform(train_labels)
y_test = encoder.transform(val_labels)

In [ ]:
num_classes = np.max(y_train) + 1
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

# Building the model

In [ ]:
batch_size = 128
epochs = 20

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               512512    
_________________________________________________________________
activation (Activation)      (None, 512)               0         
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 1539      
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 514,051
Trainable params: 514,051
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/20
47/47 [==============================] - 14s 23ms/step - loss: 1.0412 - accuracy: 0.4589 - val_loss: 0.9139 - val_accuracy: 0.6063
Epoch 2/20
47/47 [==============================] - 0s 10ms/step - loss: 0.8197 - accuracy: 0.6780 - val_loss: 0.8473 - val_accuracy: 0.6501
Epoch 3/20
47/47 [==============================] - 0s 10ms/step - loss: 0.6823 - accuracy: 0.7344 - val_loss: 0.8198 - val_accuracy: 0.6546
Epoch 4/20
47/47 [==============================] - 0s 10ms/step - loss: 0.5875 - accuracy: 0.7745 - val_loss: 0.8237 - val_accuracy: 0.6591
Epoch 5/20
47/47 [==============================] - 0s 10ms/step - loss: 0.5145 - accuracy: 0.7981 - val_loss: 0.8395 - val_accuracy: 0.6697
Epoch 6/20
47/47 [==============================] - 0s 10ms/step - loss: 0.4608 - accuracy: 0.8309 - val_loss: 0.8497 - val_accuracy: 0.6682
Epoch 7/20
47/47 [==============================] - 0s 10ms/step - loss: 0.4051 - accuracy: 0.8604 - val_loss: 0.8693 - val_accuracy: 0.6606
Epoch 8/20
4

In [ ]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

13/13 [==============================] - 0s 4ms/step - loss: 1.1100 - accuracy: 0.6673
Test accuracy: 0.6672705411911011
